In [1]:
import tensorflow as tf
import os

**load the data**

In [2]:
currentDir = os.getcwd()

filePath = os.path.join(currentDir, "Flower Classification Dataset/train")

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    filePath,
    labels = 'inferred',
    image_size = (224,224),
    batch_size = 102,
    validation_split = 0
)

Found 4102 files belonging to 102 classes.


**Outline the model**

In [3]:
input_size = 50176
output_size = 102
hidden_layer_size = 128 #TODO: figure out the exact value

model = tf.keras.Sequential([
    # ReScale pixel values in the band [0, 1] 
    tf.keras.Input(shape=(224, 224, 3)),
    tf.keras.layers.Rescaling(1./255),

    # Augmentation (Adding randomization to the images)
    # ONLY USED IN TRAINING
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.2),
    
    # Conv 1
    tf.keras.layers.Conv2D(
        32, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ), #TODO: Check if the combination of L1 and L2 works better for all layers
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 2
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 3
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    # Conv 4
    tf.keras.layers.Conv2D(
        128, 
        (3,3), 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    # Flattened Inputs
    tf.keras.layers.GlobalAveragePooling2D(), # Better than normal Flattening, reduces the dimensionality of feature maps
        #tf.keras.layers.Flatten(),
    
    # Dense Layers (Fully connected layers (FC Layers))
    tf.keras.layers.Dense(
        hidden_layer_size, 
        activation = 'relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    #TODO: Check if dropping neurons actually prevents overfitting 
    #TODO: We can test other techniques besides dropping 
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(        
        hidden_layer_size, 
        activation = 'relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        bias_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.Dropout(0.3),

    # Didn't specify "softmax" activation function in the last layer 
    # to use the right order of opertions for loss calculations
    tf.keras.layers.Dense(output_size) 
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast                 │ (None, 224, 224, 3)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 222, 222, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 109, 109, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 52, 52, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             

 Total params: 177,510 (693.40 KB)

 Trainable params: 176,934 (691.15 KB)

 Non-trainable params: 576 (2.25 KB)

In [9]:
lossFunction = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) # Internally applies "softmax" if "from_logits = True"

model.compile(
    loss=lossFunction, 
    label_smoothing = 0.1 #TODO: Test other values 
    )

TypeError: Trainer.compile() got an unexpected keyword argument 'label_smoothing'